In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from __future__ import print_function
from skimage.io import imread, imsave
import argparse
import numpy as np
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
import torch.backends.cudnn as cudnn
from torchvision.utils import save_image
from net import MaskGenerator, ResiduePredictor
from color_cnn import ColorCNN
import cv2
import os
import time
from sklearn.cluster import KMeans
import pandas as pd
from guided_filter_pytorch.guided_filter import GuidedFilter
from mydataset import MyDataset

In [21]:
#### User inputs

run_name = 'sample'
num_primary_color = 7
csv_path = 'sample.csv' # なんでも良い．後方でパスを置き換えるから
resize_scale_factor = 1 

img_name = 'test4'
img_path = 'G:/FSCS-master/dataset/test/' + img_name

path_plan2_generator = 'results/' + run_name + '/plan2_generator.pth'
path_mask_generator = 'results/' + run_name + '/mask_generator.pth'
path_residue_predictor = 'results/' + run_name + '/residue_predictor.pth'

try:
    os.makedirs('results/%s/%s' % (run_name, img_name))
except OSError:
    pass

In [22]:
test_dataset = MyDataset(csv_path, num_primary_color, mode='test')
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    )

device = 'cuda'

# define model
plan2_generator = ColorCNN('unet', 7 ).to(device)
mask_generator = MaskGenerator(num_primary_color).to(device)
residue_predictor = ResiduePredictor(num_primary_color).to(device)


# load params
plan2_generator.load_state_dict(torch.load(path_plan2_generator))
mask_generator.load_state_dict(torch.load(path_mask_generator))
residue_predictor.load_state_dict(torch.load(path_residue_predictor))


# eval mode
plan2_generator.eval()
mask_generator.eval()
residue_predictor.eval()

ResiduePredictor(
  (conv1): Conv2d(31, 62, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (conv2): Conv2d(62, 124, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (conv3): Conv2d(124, 248, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (deconv1): ConvTranspose2d(248, 124, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1), bias=False)
  (deconv2): ConvTranspose2d(248, 62, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1), bias=False)
  (deconv3): ConvTranspose2d(124, 62, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1), bias=False)
  (conv4): Conv2d(65, 31, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(31, 21, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(62, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(124, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3

In [23]:
# 必要な関数を定義する

def replace_color(primary_color_layers, manual_colors):
    temp_primary_color_layers = primary_color_layers.clone()
    for layer in range(len(manual_colors)):
        for color in range(3):
                temp_primary_color_layers[:,layer,color,:,:].fill_(manual_colors[layer][color])
    return temp_primary_color_layers

def cut_edge(target_img):
    #print(target_img.size())
    target_img = F.interpolate(target_img, scale_factor=resize_scale_factor, mode='area')
    #print(target_img.size())
    h = target_img.size(2)
    w = target_img.size(3)
    h = h - (h % 8)
    w = w - (w % 8)
    target_img = target_img[:,:,:h,:w]
    #print(target_img.size())
    return target_img

def alpha_normalize(alpha_layers):
    # constraint (sum = 1)
    # layersの状態で受け取り，その形で返す. bn, ln, 1, h, w
    return alpha_layers / (alpha_layers.sum(dim=1, keepdim=True) + 1e-8)

def read_backimage():
    img = cv2.imdecode(np.fromfile('G:/FSCS-master/dataset/backimage.jpg', dtype = np.uint8), -1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.transpose((2,0,1))
    img = img/255
    img = torch.from_numpy(img.astype(np.float32))

    return img.view(1,3,256,256).to(device)

backimage = read_backimage()

def proc_guidedfilter(alpha_layers, guide_img):
    # guide_imgは， 1chのモノクロに変換
    # target_imgを使う． bn, 3, h, w
    guide_img = (guide_img[:, 0, :, :]*0.299 + guide_img[:, 1, :, :]*0.587 + guide_img[:, 2, :, :]*0.114).unsqueeze(1)
        
    # lnのそれぞれに対してguideg filterを実行
    for i in range(alpha_layers.size(1)):
        # layerは，bn, 1, h, w
        layer = alpha_layers[:, i, :, :, :]
        
        processed_layer = GuidedFilter(3, 1*1e-6)(guide_img, layer)
        # レイヤーごとの結果をまとめてlayersの形に戻す (bn, ln, 1, h, w)
        if i == 0: 
            processed_alpha_layers = processed_layer.unsqueeze(1)
        else:
            processed_alpha_layers = torch.cat((processed_alpha_layers, processed_layer.unsqueeze(1)), dim=1)
    
    return processed_alpha_layers

In [27]:
# test_dataset.imgs_path[0] = "G:/val_large_slices2/test_img-00_target_img.png"
# test_dataset.imgs_path[0] = "G:/val_large_slices2/img_59.jpg"
test_dataset.imgs_path[0] = "G:/FSCS-master/img/2015.tif"
# test_dataset.imgs_path[0] = "G:/voc2012/VOC2012/JPEGImages/2008_000358.jpg"

In [28]:
print('Start!')
img_number = 0

mean_estimation_time = 0
with torch.no_grad():
    for batch_idx, (target_img, primary_color_layers) in enumerate(test_loader):
        print(primary_color_layers.size())
        if batch_idx != img_number:
            print('Skip ', batch_idx)
            continue
        print('img #', batch_idx)
        target_img = cut_edge(target_img)
        target_img = target_img.to(device) # bn, 3ch, h, w
        primary_color_layers = primary_color_layers.to(device)
        #primary_color_layers = color_regresser(target_img)
        ##
        ##
        # primary_color_layers = replace_color(primary_color_layers, manual_colors) #ここ
        a,b,primary_color_layers = plan2_generator(target_img) # size(1, 3, 7, 1, 1)
        print('primary_color_layers: ', primary_color_layers.shape)
        primary_color_layers1 = primary_color_layers.repeat(primary_color_layers.size(0), 1,1 , target_img.size(2), target_img.size(3)) # size(1, 3, 7, 256, 256)
        print('primary_color_layers1: ', primary_color_layers1.shape)
        #print(primary_color_layers.size())
        start_time = time.time()
        primary_color_pack = primary_color_layers1.view(primary_color_layers1.size(0), -1 , primary_color_layers1.size(3), 
                                                        primary_color_layers1.size(4)) # size(1, 21, 256, 256)
        print('primary_color_pack: ', primary_color_pack.size())
        primary_color_pack = cut_edge(primary_color_pack) # cut edge 
        primary_color_layers = primary_color_pack.view(primary_color_pack.size(0),-1,3,primary_color_pack.size(2), 
                                                       primary_color_pack.size(3)) # size(1, 7, 3, 256, 256) cut edge 后再还原size
        print('primary_color_layers2: ', primary_color_layers.shape)
        pred_alpha_layers_pack = mask_generator(target_img, primary_color_pack)
        pred_alpha_layers = pred_alpha_layers_pack.view(target_img.size(0), -1, 1, target_img.size(2), target_img.size(3))
        ## Alpha Layer Proccessing
        processed_alpha_layers = alpha_normalize(pred_alpha_layers) 
        #processed_alpha_layers = mask_operate(processed_alpha_layers, target_layer_number, mask_path) # Option
        processed_alpha_layers = proc_guidedfilter(processed_alpha_layers, target_img) # Option
        processed_alpha_layers = alpha_normalize(processed_alpha_layers)  # Option
        ##
        mono_color_layers = torch.cat((primary_color_layers, processed_alpha_layers), 2) #shape: bn, ln, 4, h, w
        mono_color_layers_pack = mono_color_layers.view(target_img.size(0), -1 , target_img.size(2), target_img.size(3))
        residue_pack  = residue_predictor(target_img, mono_color_layers_pack)
        residue = residue_pack.view(target_img.size(0), -1, 3, target_img.size(2), target_img.size(3))
        pred_unmixed_rgb_layers = torch.clamp((primary_color_layers + residue), min=0., max=1.0)
        print("==============================")
        print("pred_unmixed_rgb_layers的size：")
        print(pred_unmixed_rgb_layers.size())
        print("==============================")
        print("processed_alpha_layers的size：")
        print(processed_alpha_layers.size())
        reconst_imggg = (pred_unmixed_rgb_layers * processed_alpha_layers)
        reconst_img = (pred_unmixed_rgb_layers * processed_alpha_layers).sum(dim=1)
        pred_unmixed_rgb_layers2016 = pred_unmixed_rgb_layers.clone()
        processed_alpha_layers2016 = processed_alpha_layers.clone()
        print("==============================")
        print("reconst_img的size：")
        print(reconst_img.size())
        print("==============================")
        print("reconst_imggg的size：")
        print(reconst_imggg.size())
        end_time = time.time()
        estimation_time = end_time - start_time
        print(estimation_time)
        mean_estimation_time += estimation_time
#                 # 処理まえのアルファを保存
#         for i in range(len(pred_alpha_layers[0])):
#             save_image(pred_alpha_layers[0,i, :, :, :], 'results/%s/%s/pred-alpha-00_layer-%02d.png' % (run_name, img_name, i) )

#         # 処理後のアルファの保存 processed_alpha_layers
#         for i in range(len(processed_alpha_layers[0])):
#             save_image(processed_alpha_layers2[0,i, :, :, :], 'results/%s/%s/test/aproc-alpha-00_layer-%02d.png' % (run_name, img_name, i))

#         # 処理後のRGBの保存
#         for i in range(len(pred_unmixed_rgb_layers[0])):
#             save_image(pred_unmixed_rgb_layers2[0,i, :, :, :], 'results/%s/%s/test/argb-00_layer-%02d.png' % (run_name, img_name, i))
        
        if True:
            # batchsizeは１で計算されているはず．それぞれ保存する．
            save_layer_number = 0
            save_image(primary_color_layers[save_layer_number,:,:,:,:],
                   'results/%s/%s/test' % (run_name, img_name) + '_img-%02d_primary_color_layers.png' % batch_idx)
            save_image(reconst_img[save_layer_number,:,:,:].unsqueeze(0),
                   'results/%s/%s/test' % (run_name, img_name) + '_img-%02d_reconst_img.png' % batch_idx)
            save_image(target_img[save_layer_number,:,:,:].unsqueeze(0),
                   'results/%s/%s/test' % (run_name, img_name)  + '_img-%02d_target_img.png' % batch_idx)

            # RGBAの４chのpngとして保存する
            RGBA_layers = torch.cat((pred_unmixed_rgb_layers, processed_alpha_layers), dim=2) # out: bn, ln, 4, h, w
            # test ではバッチサイズが１なので，bn部分をなくす
            RGBA_layers = RGBA_layers[0] # ln, 4. h, w
            # ln ごとに結果を保存する
            for i in range(len(RGBA_layers)):
                save_image(RGBA_layers[i, :, :, :], 'results/%s/%s/img-%02d_layer-%02d.png' % (run_name, img_name, batch_idx, i) )
            print('Saved to results/%s/%s/...' % (run_name, img_name))
            
        if False:
            ### mono_colorの分も保存する ###
            # RGBAの４chのpngとして保存する
            mono_RGBA_layers = torch.cat((primary_color_layers, processed_alpha_layers), dim=2) # out: bn, ln, 4, h, w
            # test ではバッチサイズが１なので，bn部分をなくす
            mono_RGBA_layers = mono_RGBA_layers[0] # ln, 4. h, w
            # ln ごとに結果を保存する
            for i in range(len(mono_RGBA_layers)):
                save_image(mono_RGBA_layers[i, :, :, :], 'results/%s/%s/mono_img-%02d_layer-%02d.png' % (run_name, img_name, batch_idx, i) )

            save_image((primary_color_layers * processed_alpha_layers).sum(dim=1)[save_layer_number,:,:,:].unsqueeze(0),
                   'results/%s/%s/test' % (run_name, img_name)  + '_mono_img-%02d_reconst_img.png' % batch_idx)   
        
        
        if batch_idx == 0:
            break # debug用


# 処理まえのアルファを保存
for i in range(len(pred_alpha_layers[0])):
    save_image(pred_alpha_layers[0,i, :, :, :], 'results/%s/%s/pred-alpha-00_layer-%02d.png' % (run_name, img_name, i) )
    
# 処理後のアルファの保存 processed_alpha_layers
for i in range(len(processed_alpha_layers[0])):
    save_image(processed_alpha_layers[0,i, :, :, :], 'results/%s/%s/proc-alpha-00_layer-%02d.png' % (run_name, img_name, i) )
#     save_image(processed_alpha_layers2[0,i, :, :, :], 'results/%s/%s/test/proc-alpha-00_layer-%02d.png' % (run_name, img_name, i) )
    
# 処理後のRGBの保存
for i in range(len(pred_unmixed_rgb_layers[0])):
    save_image(pred_unmixed_rgb_layers[0,i, :, :, :], 'results/%s/%s/rgb-00_layer-%02d.png' % (run_name, img_name, i))
#     save_image(pred_unmixed_rgb_layers2[0,i, :, :, :], 'results/%s/%s/test/rgb-00_layer-%02d.png' % (run_name, img_name, i))

Start!
torch.Size([1, 7, 3, 1760, 1872])
img # 0
primary_color_layers:  torch.Size([1, 7, 3, 1, 1])
primary_color_layers1:  torch.Size([1, 7, 3, 1760, 1872])
primary_color_pack:  torch.Size([1, 21, 1760, 1872])
primary_color_layers2:  torch.Size([1, 7, 3, 1760, 1872])
pred_unmixed_rgb_layers的size：
torch.Size([1, 7, 3, 1760, 1872])
processed_alpha_layers的size：
torch.Size([1, 7, 1, 1760, 1872])
reconst_img的size：
torch.Size([1, 3, 1760, 1872])
reconst_imggg的size：
torch.Size([1, 7, 3, 1760, 1872])
0.03900265693664551
Saved to results/sample/test4/...


In [29]:
# 処理まえのアルファを保存
for i in range(len(pred_alpha_layers[0])):
    save_image(pred_alpha_layers[0,i, :, :, :], 'results/%s/%s/pred-alpha-00_layer-%02d.png' % (run_name, img_name, i) )
    
# 処理後のアルファの保存 processed_alpha_layers
for i in range(len(processed_alpha_layers[0])):
    save_image(processed_alpha_layers[0,i, :, :, :], 'results/%s/%s/proc-alpha-00_layer-%02d.png' % (run_name, img_name, i) )
#     save_image(processed_alpha_layers2[0,i, :, :, :], 'results/%s/%s/test/proc-alpha-00_layer-%02d.png' % (run_name, img_name, i) )
    
# 処理後のRGBの保存
for i in range(len(pred_unmixed_rgb_layers[0])):
    save_image(pred_unmixed_rgb_layers[0,i, :, :, :], 'results/%s/%s/rgb-00_layer-%02d.png' % (run_name, img_name, i))
#     save_image(pred_unmixed_rgb_layers2[0,i, :, :, :], 'results/%s/%s/test/rgb-00_layer-%02d.png' % (run_name, img_name, i))

In [9]:
reconst_img2015 = (pred_unmixed_rgb_layers2015 * processed_alpha_layers2015).sum(dim=1)
save_image(reconst_img2015, 'J:/FSCS-master/results/sample/2015.jpg/reconst2015.png')

NameError: name 'pred_unmixed_rgb_layers2015' is not defined

In [13]:
reconst_img2016 = (pred_unmixed_rgb_layers2016 * processed_alpha_layers2016).sum(dim=1)
save_image(reconst_img2016, 'J:/FSCS-master/results/sample/2016.jpg/reconst2016.png')

In [25]:
reconst_img2015_2016 = (pred_unmixed_rgb_layers2016_2 * processed_alpha_layers2015).sum(dim=1)
save_image(reconst_img2015_2016, 'J:/FSCS-master/results/sample/2015.jpg/reconst2015--2016.png')

In [23]:
reconst_img2016_2015 = (pred_unmixed_rgb_layers2015_2 * processed_alpha_layers2016).sum(dim=1)
save_image(reconst_img2016_2015, 'J:/FSCS-master/results/sample/2015.jpg/reconst2016--2015.png')

In [24]:
pred_unmixed_rgb_layers2016_2 = pred_unmixed_rgb_layers2016.clone()
pred_unmixed_rgb_layers2015_2 = pred_unmixed_rgb_layers2015.clone()
pred_unmixed_rgb_layers2016_2[0,4,:,:,:] = pred_unmixed_rgb_layers2016[0,4,:,:,:]
pred_unmixed_rgb_layers2016_2[0,5,:,:,:] = pred_unmixed_rgb_layers2016[0,6,:,:,:]
pred_unmixed_rgb_layers2016_2[0,0,:,:,:] = pred_unmixed_rgb_layers2016[0,1,:,:,:]
pred_unmixed_rgb_layers2016_2[0,1,:,:,:] = pred_unmixed_rgb_layers2016[0,0,:,:,:]
pred_unmixed_rgb_layers2016_2[0,2,:,:,:] = pred_unmixed_rgb_layers2016[0,2,:,:,:]
pred_unmixed_rgb_layers2016_2[0,3,:,:,:] = pred_unmixed_rgb_layers2016[0,3,:,:,:]
pred_unmixed_rgb_layers2016_2[0,6,:,:,:] = pred_unmixed_rgb_layers2016[0,5,:,:,:]

In [21]:
pred_unmixed_rgb_layers2016_2 = pred_unmixed_rgb_layers2016.clone()
pred_unmixed_rgb_layers2015_2 = pred_unmixed_rgb_layers2015.clone()
pred_unmixed_rgb_layers2015_2[0,4,:,:,:] = pred_unmixed_rgb_layers2015[0,4,:,:,:]
pred_unmixed_rgb_layers2015_2[0,5,:,:,:] = pred_unmixed_rgb_layers2015[0,6,:,:,:]
pred_unmixed_rgb_layers2015_2[0,0,:,:,:] = pred_unmixed_rgb_layers2015[0,1,:,:,:]
pred_unmixed_rgb_layers2015_2[0,1,:,:,:] = pred_unmixed_rgb_layers2015[0,0,:,:,:]
pred_unmixed_rgb_layers2015_2[0,2,:,:,:] = pred_unmixed_rgb_layers2015[0,2,:,:,:]
pred_unmixed_rgb_layers2015_2[0,3,:,:,:] = pred_unmixed_rgb_layers2015[0,3,:,:,:]
pred_unmixed_rgb_layers2015_2[0,6,:,:,:] = pred_unmixed_rgb_layers2015[0,5,:,:,:]

In [93]:
reconst_img2015_2016 = (pred_unmixed_rgb_layers2016_2 * processed_alpha_layers2015).sum(dim=1)
save_image(reconst_img2015_2016, 'J:/FSCS-master/results/sample/2015.jpg/reconst2015--2016_2.png')

In [38]:
rgb_layer = imread('J:/FSCS-master/results/sample/apple.jpg/rgb-00_layer-01.png')
rgb_layer.shape

(816, 896, 3)

In [45]:
a = imread('J:/FSCS-master/results/sample/apple.jpg/rgb-00_layer-00.png')
b = imread('J:/FSCS-master/results/sample/apple.jpg/rgb-00_layer-01.png')
c = imread('J:/FSCS-master/results/sample/apple.jpg/rgb-00_layer-02.png')
d = imread('J:/FSCS-master/results/sample/apple.jpg/rgb-00_layer-03.png')
e = imread('J:/FSCS-master/results/sample/apple.jpg/rgb-00_layer-04.png')
f = imread('J:/FSCS-master/results/sample/apple.jpg/rgb-00_layer-05.png')
g = imread('J:/FSCS-master/results/sample/apple.jpg/rgb-00_layer-06.png')

aa = imread('J:/FSCS-master/results/sample/apple.jpg/proc-alpha-00_layer-00.png')
bb = imread('J:/FSCS-master/results/sample/apple.jpg/proc-alpha-00_layer-01.png')
cc = imread('J:/FSCS-master/results/sample/apple.jpg/proc-alpha-00_layer-02.png')
dd = imread('J:/FSCS-master/results/sample/apple.jpg/proc-alpha-00_layer-03.png')
ee = imread('J:/FSCS-master/results/sample/apple.jpg/proc-alpha-00_layer-04.png')
ff = imread('J:/FSCS-master/results/sample/apple.jpg/proc-alpha-00_layer-05.png')
gg = imread('J:/FSCS-master/results/sample/apple.jpg/proc-alpha-00_layer-06.png')

In [49]:
rgb = np.array([aa[:,:,0],bb[:,:,0],cc[:,:,0],dd[:,:,0],ee[:,:,0],ff[:,:,0],gg[:,:,0]])
alpha = np.array([a,b,c,d,e,f,g])
alpha = alpha.transpose((0,3,1,2))

In [54]:
print(alpha.shape)
print(rgb.shape)

(7, 3, 816, 896)
(7, 816, 896)


In [55]:
x = torch.from_numpy(alpha)
y = torch.from_numpy(rgb)
torch_alpha = x.unsqueeze(0)
torch_rgb = y.unsqueeze(0).unsqueeze(2)

In [56]:
print(torch_alpha.shape)
print(torch_rgb.shape)

torch.Size([1, 7, 3, 816, 896])
torch.Size([1, 7, 1, 816, 896])


In [71]:
reconst = (torch_alpha*torch_rgb).sum(dim = 1)
save_image(reconst, 'J:/FSCS-master/results/sample/apple.jpg/test/reconst02.png')

In [23]:
im2015 = np.array([[128, 120, 113],
 [ 16,  24,  41],
 [181, 170, 161],
 [154, 144, 137],
 [ 60,  62,  66],
 [100,  96,  92],
 [205, 197, 187]]
)
im2016 = np.array([[192, 185, 172],
 [ 43,  54,  46],
 [ 18,  29,  25],
 [136, 124, 111],
 [163, 151, 137],
 [111,  97,  85],
 [ 84,  71,  58]]
)
im2015_2016 = np.array([[116, 107,  98],
                    [145, 134, 124],
                    [ 17,  26,  35],
                    [175, 164, 153],
                    [ 48,  55,  49],
                    [ 91,  80,  70],
                    [201, 193, 183]]
)

In [24]:
im2015.shape

(7, 3)

In [25]:
distance11 = []
distance22 = []
for i in range(7):
    distance1 = []
    distance2 = []
    for j in range(7):
        dis1 = np.sqrt(np.sum(np.square(im2015_2016[i] - im2015[j])))
        distance1.append(dis1)
        dis2 = np.sqrt(np.sum(np.square(im2015_2016[i] - im2016[j])))
        distance2.append(dis2)
    distance11.append(distance1)
    distance22.append(distance2)

In [26]:
np.array(distance22).shape

(7, 7)

In [27]:
distance11

[[23.194827009486403,
  141.90842117365693,
  110.28599185753374,
  65.83312236253116,
  78.64477096412705,
  20.322401432901575,
  154.7320264198721],
 [24.61706725018234,
  188.7591057406238,
  62.93647591023825,
  18.708286933869708,
  125.59060474414477,
  67.02984409947557,
  107.41508273980894],
 [165.04847772699995,
  6.4031242374328485,
  252.00793638296395,
  207.59816954877036,
  64.07807737440318,
  122.62952336203546,
  296.12328513644445],
 [75.79577824654879,
  239.63513932643517,
  11.661903789690601,
  33.12099032335839,
  176.62955585065598,
  118.19475453673907,
  56.08029957123981],
 [121.33012816279393,
  45.2658811910251,
  208.46582453726077,
  164.01524319403975,
  21.95449840010015,
  78.95568377260753,
  252.69942619642015],
 [69.4118145563131,
  97.98979538707079,
  156.46405337968207,
  112.0446339634344,
  36.069377593742864,
  28.653097563788805,
  200.93282459568422],
 [124.73171208638162,
  288.0104164782934,
  37.589892258425,
  82.0121942152507,
  225.2

In [28]:
distance22

[[131.6662447250623,
  104.12492496996096,
  144.97241116847025,
  29.29163703175362,
  75.27283706623525,
  17.146428199482248,
  62.609903369994115],
 [84.34453153583817,
  151.28780519261954,
  192.23683309917485,
  18.708286933869708,
  27.964262908219126,
  63.608175575157006,
  109.754270987511],
 [273.2672684387942,
  39.761790704142086,
  10.488088481701515,
  171.87495454544853,
  217.5890622251036,
  127.97265332874832,
  83.92258337301111],
 [33.03028912982749,
  202.41788458533006,
  243.4296612987004,
  69.8927750200262,
  23.853720883753127,
  114.9304137293519,
  161.1055554597668],
 [229.7063342618135,
  5.916079783099616,
  46.389654018972806,
  127.8632081562167,
  173.738308959193,
  83.83913167489273,
  40.4103947023535],
 [177.8482499211055,
  59.632206063502295,
  99.77474630386187,
  75.11324783285569,
  121.30127781684742,
  30.23243291566195,
  16.55294535724685],
 [16.30950643030009,
  251.10555549409892,
  292.1455116889527,
  119.03780911962384,
  72.9657453

In [58]:
from skimage.exposure import match_histograms

In [61]:
image = imread('G:/FSCS-master/results/sample/2015_6-5/rgb-00_layer-00.png')
reference = imread('G:/FSCS-master/results/sample/2016_6-5/rgb-00_layer-06.png')
matched0 = match_histograms(image, reference, multichannel=True)/255

image = imread('G:/FSCS-master/results/sample/2015_6-5/rgb-00_layer-01.png')
reference = imread('G:/FSCS-master/results/sample/2016_6-5/rgb-00_layer-02.png')
matched1 = match_histograms(image, reference, multichannel=True)/255

image = imread('G:/FSCS-master/results/sample/2015_6-5/rgb-00_layer-02.png')
reference = imread('G:/FSCS-master/results/sample/2016_6-5/rgb-00_layer-04.png')
matched2 = match_histograms(image, reference, multichannel=True)/255

image = imread('G:/FSCS-master/results/sample/2015_6-5/rgb-00_layer-03.png')
reference = imread('G:/FSCS-master/results/sample/2016_6-5/rgb-00_layer-03.png')
matched3 = match_histograms(image, reference, multichannel=True)/255

image = imread('G:/FSCS-master/results/sample/2015_6-5/rgb-00_layer-04.png')
reference = imread('G:/FSCS-master/results/sample/2016_6-5/rgb-00_layer-01.png')
matched4 = match_histograms(image, reference, multichannel=True)/255

image = imread('G:/FSCS-master/results/sample/2015_6-5/rgb-00_layer-05.png')
reference = imread('G:/FSCS-master/results/sample/2016_6-5/rgb-00_layer-05.png')
matched5 = match_histograms(image, reference, multichannel=True)/255

image = imread('G:/FSCS-master/results/sample/2015_6-5/rgb-00_layer-06.png')
reference = imread('G:/FSCS-master/results/sample/2016_6-5/rgb-00_layer-00.png')
matched6 = match_histograms(image, reference, multichannel=True)/255


In [62]:
alpha0 = imread('G:/FSCS-master/results/sample/2015_6-5/proc-alpha-00_layer-00.png')/255
alpha1 = imread('G:/FSCS-master/results/sample/2015_6-5/proc-alpha-00_layer-01.png')/255
alpha2 = imread('G:/FSCS-master/results/sample/2015_6-5/proc-alpha-00_layer-02.png')/255
alpha3 = imread('G:/FSCS-master/results/sample/2015_6-5/proc-alpha-00_layer-03.png')/255
alpha4 = imread('G:/FSCS-master/results/sample/2015_6-5/proc-alpha-00_layer-04.png')/255
alpha5 = imread('G:/FSCS-master/results/sample/2015_6-5/proc-alpha-00_layer-05.png')/255
alpha6 = imread('G:/FSCS-master/results/sample/2015_6-5/proc-alpha-00_layer-06.png')/255

In [63]:
reconst0 = np.around(alpha0*matched0*255).astype(np.uint8)
reconst1 = np.around(alpha1*matched1*255).astype(np.uint8)
reconst2 = np.around(alpha2*matched2*255).astype(np.uint8)
reconst3 = np.around(alpha3*matched3*255).astype(np.uint8)
reconst4 = np.around(alpha4*matched4*255).astype(np.uint8)
reconst5 = np.around(alpha5*matched5*255).astype(np.uint8)
reconst6 = np.around(alpha6*matched6*255).astype(np.uint8)
img = reconst0+reconst1+reconst2+reconst3+reconst4+reconst5+reconst6
imsave('G:/FSCS-master/results/sample/2015_6-5/reconst2015-2016.png', img)

In [1]:
import torch

In [10]:
x = torch.ones(1, 7, 256, 256)
y = torch.ones(1, 1, 256, 256).item()

#tensor([[0.1940, 0.3340, 0.8184, 0.4269, 0.5945],
#        [0.2078, 0.5978, 0.0074, 0.0943, 0.0266]])

torch.zeros(1, 7, 256, 256).scatter(1, y, 1)


#tensor([[0.1940, 0.5978, 0.0074, 0.4269, 0.5945],
#        [0.0000, 0.3340, 0.0000, 0.0943, 0.0000],
#        [0.2078, 0.0000, 0.8184, 0.0000, 0.0266]])

ValueError: only one element tensors can be converted to Python scalars

In [6]:
x = torch.ones((8,7,3,1,1))*9
y = x.repeat(1, 1, 1, 256, 256)

In [7]:
y.size()

torch.Size([8, 7, 3, 256, 256])

In [8]:
y

tensor([[[[[9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.],
           ...,
           [9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.]],

          [[9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.],
           ...,
           [9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.]],

          [[9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.],
           ...,
           [9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.]]],


         [[[9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.],
           [9., 9., 9.,  ..., 9., 9., 9.],
           ...,
           [9., 9., 9.,  